In [1]:
import pyodbc
import pandas as pd
import numpy as np
import os
import sqlite3
from datetime import datetime, timezone
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
df_already_scanned = pd.read_csv("already_scanned.csv", sep=";")

In [160]:
paramList = []
entry = []
x = len(df_already_scanned)

In [161]:
entry.append('sumRecord_Number')
entry.append(df_already_scanned['sourceRecordNumber'].sum())
paramList.append(entry)
entry = []
entry.append('minSumPriceBill')
entry.append(df_already_scanned['dfSumPriceBill'].min())
paramList.append(entry)
entry = []
entry.append('maxSumPriceBill')
entry.append(df_already_scanned['dfSumPriceBill'].max())
paramList.append(entry)
entry = []
entry.append('avgSumPriceBill')
entry.append(df_already_scanned['dfSumPriceBill'].mean())
paramList.append(entry)
entry = []

In [163]:
penultSumPriceBill = df_already_scanned.loc[x-2:x-2, ['dfSumPriceBill']].iloc[0,0]
entry.append('penultSumPriceBill')
entry.append(penultSumPriceBill)
paramList.append(entry)
entry = []

lastSumPriceBill = df_already_scanned.loc[x-1:x, ['dfSumPriceBill']].iloc[0,0]
entry.append('lastSumPriceBill')
entry.append(lastSumPriceBill)
paramList.append(entry)
entry = []

changeLastSumPriceBill = ((lastSumPriceBill/penultSumPriceBill)-1)*100
entry.append('changeLastSumPriceBill')
entry.append(changeLastSumPriceBill)
paramList.append(entry)
entry = []

penultSumBoxMill = df_already_scanned.loc[x-2:x-2, ['dfSumBoxMill']].iloc[0,0]
entry.append('penultSumBoxMill')
entry.append(penultSumBoxMill)
paramList.append(entry)
entry = []

lastSumBoxMill = df_already_scanned.loc[x-1:x, ['dfSumBoxMill']].iloc[0,0]
entry.append('lastSumBoxMill')
entry.append(lastSumBoxMill)
paramList.append(entry)
entry = []

changeLastSumBoxMill = ((lastSumBoxMill/penultSumBoxMill)-1)*100
entry.append('changeLastSumBoxMill')
entry.append(changeLastSumBoxMill)
paramList.append(entry)
entry = []

yesterSumBoxMill = df_already_scanned.loc[x-13:x-12, ['dfSumBoxMill']].iloc[0,0]
entry.append('yesterSumBoxMill')
entry.append(yesterSumBoxMill)
paramList.append(entry)
entry = []

changeYesterSumBoxMill = ((lastSumBoxMill/yesterSumBoxMill)-1)*100
entry.append('changeYesterSumBoxMill')
entry.append(changeYesterSumBoxMill)
paramList.append(entry)
entry = []


yesterSumPriceBill = df_already_scanned.loc[x-13:x-12, ['dfSumPriceBill']].iloc[0,0]
entry.append('yesterSumPriceBill')
entry.append(yesterSumPriceBill)
paramList.append(entry)
entry = []


changeYesterSumPriceBill = ((lastSumPriceBill/yesterSumPriceBill)-1)*100
entry.append('changeYesterSumPriceBill')
entry.append(changeYesterSumPriceBill)
paramList.append(entry)
entry = []

In [165]:
neak_paramList = pd.DataFrame(paramList, columns =['parameter', 'value'])

In [166]:
neak_paramList.to_csv("neak_paramList.csv", sep=";", encoding='utf-8', index=False)

In [180]:
# Abbreviations of the Names of the Months (Yale University Library)
# https://web.library.yale.edu/cataloging/months
# line = "English"

monthDict = {
'01':"Jan.",
'02':"Feb.",
'03':"Mar.",
'04':"Apr.",
'05':"May",
'06':"June",
'07':"July",
'08':"Aug.",
'09':"Sept.",
'10':"Oct.",
'11':"Nov.",
'12':"Dec."}

In [2]:
db = sqlite3.connect('neak2.sqlite3')

In [ ]:
# q = "select * from records_number"
# q = "select * from ttt_db"
# q = "select * from atc1"
# q = "select * from atc3"
# q = "select * from sqlite_schema where type like 'table'"
# q = "select * from sqlite_schema where type like 'view'"
# q = "select * from recent5"

# q = "select idoszak, sum(fogyar) as sumfogyar, sum(doboz) as sumdoboz from recent6 group by idoszak;"

In [170]:
q = "select idoszak, sum(fogyar) as sumfogyar, sum(doboz) as sumdoboz from recent6 group by idoszak;"

In [171]:
df = pd.read_sql(q, db)

In [176]:
df['year']= df['idoszak'].str.slice(0, 4)

In [177]:
df['honap']= df['idoszak'].str.slice(4, 6)

In [181]:
df['month'] = df['honap'].map(monthDict)

In [183]:
df.to_csv("recent6_monthly.csv", sep=";", encoding='utf-8', index=False)

In [36]:
q = "select * from years;"

In [37]:
df = pd.read_sql(q, db)

In [39]:
df.to_csv("neak_years.csv", sep=";", encoding='utf-8', index=False)

In [3]:
q = "SELECT idoszak, substr(atc, 1, 1) AS atc1, sum(doboz) AS sumdob, sum(fogyar) AS sumfogyar FROM whole2022 GROUP BY substr(atc, 1, 1), idoszak;"

In [4]:
df = pd.read_sql(q, db)

In [22]:
df['changeSumDob'] = (((df['sumdob']/df.shift(1, axis = 0)['sumdob'])-1)*100).round(decimals=2)

In [23]:
df['changeSumFogyar'] = (((df['sumfogyar']/df.shift(1, axis = 0)['sumfogyar'])-1)*100).round(decimals=2)

In [25]:
df.to_csv("neak_atc1.csv", sep=";", encoding='utf-8', index=False)

In [26]:
q = "SELECT idoszak, megye, sum(doboz) AS sumdob, sum(fogyar) AS sumfogyar FROM whole2022 GROUP BY idoszak, megye;"

In [27]:
df = pd.read_sql(q, db)

In [29]:
# https://www.google.com/url?sa=t&rct=j&q=&esrc=s&source=web&cd=&ved=2ahUKEwjki_OS-5v5AhVN2aQKHajIDVkQFnoECBEQAQ&url=https%3A%2F%2Fwww.ksh.hu%2Fdocs%2Fhun%2Fxftp%2Froviditesek.pdf&usg=AOvVaw2_-SdIQNO8KCskUhnIm9gd
# KSH módszertani útmutató
# Megyék rövidítésjegyzéke

megyedict = {"BA":"Baranya",
"BO":"Borsod-Abaúj-Zemplén",
"BP":"Budapest",
"BÁ":"Bács-Kiskun",
"BÉ":"Békés",
"CS":"Csongrád-Csanád",
"FE":"Fejér",
"GY":"Győr-Moson-Sopron",
"HA":"Hajdú-Bihar",
"HE":"Heves",
"JÁ":"Jász-Nagykun-Szolnok",
"KO":"Komárom-Esztergom",
"NÓ":"Nógrád",
"PE":"Pest",
"SO":"Somogy",
"SZ":"Szabolcs-Szatmár-Bereg",
"TO":"Tolna",
"VA":"Vas",
"VE":"Veszprém",
"ZA":"Zala"}

In [30]:
df['megyekod'] = df['megye']

In [31]:
df['megye'] = df['megye'].map(megyedict)

In [36]:
df.sort_values(by=['megye','idoszak'], inplace=True)

In [37]:
df['changeSumDob'] = (((df['sumdob']/df.shift(1, axis = 0)['sumdob'])-1)*100).round(decimals=2)

In [38]:
df['changeSumFogyar'] = (((df['sumfogyar']/df.shift(1, axis = 0)['sumfogyar'])-1)*100).round(decimals=2)

In [42]:
df.to_csv("neak_megye.csv", sep=";", encoding='utf-8', index=False)

In [43]:
db.close()